In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
from wordcloud import WordCloud
import numpy as np
from sklearn import metrics
import seaborn as sns
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import nltk
nltk.download('stopwords')
from sklearn.decomposition import LatentDirichletAllocation
from nltk.stem import WordNetLemmatizer #word stemmer class
lemma = WordNetLemmatizer()
from nltk.sentiment.vader import SentimentIntensityAnalyzer as VS


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
tweet1 = pd.read_csv('/content/drive/Shareddrives/Text mining Project/PROJECT/TWEET 1.csv')
tweet2 = pd.read_csv('/content/drive/Shareddrives/Text mining Project/PROJECT/TWEET 2.csv')

In [ ]:
# Distribution of dataset
tweet1.groupby('class').count()

,Unnamed: 0,count,hate_speech,offensive_language,neither,tweet
class,,,,,,
0,1430,1430,1430,1430,1430,1430
1,19190,19190,19190,19190,19190,19190
2,4163,4163,4163,4163,4163,4163


In [ ]:
# Distribution of dataset
tweet2.groupby('label').count()

,Unnamed: 0,id,text,type,model_wrong,db.model_preds,status,round,split,annotator
label,,,,,,,,,,
hate,22124,22124,22124,22124,15253,15253,22124,22124,22124,22124
nothate,18499,18499,18499,18499,10844,10844,18499,18499,18499,18499


NAIVE BAYES

In [ ]:
X2 = tweet2['text']
y2 = tweet2['label']

# Splitting the data into training and test sets
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.3, random_state=42)

# Vectorizing the text data
vectorizer2 = CountVectorizer(
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.8,
    stop_words='english',
    max_features=5000,
    binary=True
)

X2_train_vectorized = vectorizer2.fit_transform(X2_train)
X2_test_vectorized = vectorizer2.transform(X2_test)

# Creating and training the Naïve Bayes model
nb2 = MultinomialNB()
nb2.fit(X2_train_vectorized, y2_train)

# Making predictions on the test set
y2_pred = nb2.predict(X2_test_vectorized)

# Evaluating the model
print("Evaluation for tweet2:")
print(metrics.classification_report(y2_test, y2_pred))
print("Accuracy:", metrics.accuracy_score(y2_test, y2_pred))

conf_matrix2 = confusion_matrix(y2_test, y2_pred)

# True Positive (TP)
tp1 = conf_matrix2[1, 1]

# True Negative (TN)
tn1 = conf_matrix2[0, 0]

# False Positive (FP)
fp1 = conf_matrix2[0, 1]

# False Negative (FN)
fn1 = conf_matrix2[1, 0]

# Sensitivity, Specificity
sensitivity2 = tp1 / (tp1 + fn1)
specificity2 = tn1 / (tn1 + fp1)

# Print the results
print("Confusion Matrix for data:")
print(conf_matrix2)
print("\nSensitivity for data:", sensitivity2)
print("Specificity for data:", specificity2)

Evaluation for tweet2:
              precision    recall  f1-score   support

        hate       0.69      0.72      0.70      6636
     nothate       0.65      0.61      0.63      5551

    accuracy                           0.67     12187
   macro avg       0.67      0.66      0.67     12187
weighted avg       0.67      0.67      0.67     12187

Accuracy: 0.6695659309099861
Confusion Matrix for data:
[[4786 1850]
 [2177 3374]]

Sensitivity for data: 0.6078184110970997
Specificity for data: 0.7212176009644364


In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'alpha': [0.1, 0.5,1.5],  # Extend the range of alpha values
    'fit_prior': [True, False],
    'class_prior': [None, [0.3, 0.7], [0.5, 0.5]],
}


# Create a GridSearchCV object
grid_search = GridSearchCV(MultinomialNB(), param_grid, cv=5, scoring='accuracy')

# Fit the grid search to the data
grid_search.fit(X2_train_vectorized, y2_train)

# Print the best parameters
print("Best Parameters:", grid_search.best_params_)

# Use the best model for prediction
best_model = grid_search.best_estimator_
y2_pred_tuned = best_model.predict(X2_test_vectorized)

# Print the evaluation metrics for the tuned model
print("Tuned Model Evaluation:")
print(metrics.classification_report(y2_test, y2_pred_tuned))
print("Tuned Model Accuracy:", metrics.accuracy_score(y2_test, y2_pred_tuned))


Best Parameters: {'alpha': 1.5, 'class_prior': None, 'fit_prior': False}
Tuned Model Evaluation:
              precision    recall  f1-score   support

        hate       0.71      0.68      0.70      6636
     nothate       0.64      0.67      0.65      5551

    accuracy                           0.68     12187
   macro avg       0.67      0.67      0.67     12187
weighted avg       0.68      0.68      0.68     12187

Tuned Model Accuracy: 0.6753097562976943


RANDOM FOREST

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier

# Use a TF-IDF vectorizer instead of CountVectorizer
vectorizer = TfidfVectorizer(
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.8,
    stop_words='english',
    max_features=5000
)

# Create a pipeline with TF-IDF vectorizer and Multinomial Naive Bayes
pipeline = make_pipeline(vectorizer, MultinomialNB(alpha=1.5, fit_prior=False))

# Train the model using the entire dataset (no need to split again for tuning)
pipeline.fit(X2, y2)

# Make predictions on the test set
y_pred_tuned = pipeline.predict(X2_test)

# Evaluate the tuned model
print("Tuned Model Evaluation:")
print(metrics.classification_report(y2_test, y_pred_tuned))
print("Tuned Model Accuracy:", metrics.accuracy_score(y2_test, y_pred_tuned))


Tuned Model Evaluation:
              precision    recall  f1-score   support

        hate       0.77      0.71      0.74      6636
     nothate       0.68      0.75      0.71      5551

    accuracy                           0.73     12187
   macro avg       0.73      0.73      0.73     12187
weighted avg       0.73      0.73      0.73     12187

Tuned Model Accuracy: 0.726593911545089


In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for SVM
param_grid_svm = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto'],
    'degree': [2, 3, 4]
}

# Create a GridSearchCV object for SVM
grid_search_svm = GridSearchCV(SVC(random_state=42), param_grid_svm, cv=3, scoring='accuracy')

# Fit the grid search to the data
grid_search_svm.fit(X2_train_vectorized, y2_train)

# Print the best parameters for SVM
print("Best Parameters for SVM:", grid_search_svm.best_params_)

# Use the best SVM model for prediction
best_model_svm = grid_search_svm.best_estimator_
y_pred_svm_tuned = best_model_svm.predict(X2_test_vectorized)

# Print the evaluation metrics for the tuned SVM model
print("Tuned SVM Model Evaluation:")
print(metrics.classification_report(y2_test, y_pred_svm_tuned))
print("Tuned SVM Model Accuracy:", metrics.accuracy_score(y2_test, y_pred_svm_tuned))